In [0]:
import scipy.io.wavfile as wav
import librosa as lr
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.listdir(data_dir)

NameError: ignored

In [0]:
labels={'tr_mini':1,'mn_mini':2,'dv_mini':3,'br_mini':4}

In [0]:
data_dir='/content/drive/My Drive/'

In [0]:
tr_mini=[data_dir+'tr_mini/'+i for i in os.listdir(data_dir+'tr_mini')]
br_mini=[data_dir+'br_mini/'+i for i in os.listdir(data_dir+'br_mini')]
mn_mini=[data_dir+'mn_mini/'+i for i in os.listdir(data_dir+'mn_mini')]
dv_mini=[data_dir+'dv_mini/'+i for i in os.listdir(data_dir+'dv_mini')]

In [6]:
data=pd.DataFrame({'fname':tr_mini,'label': [1 for i in range(160)]})

ValueError: ignored

In [7]:
data=pd.concat([data,pd.DataFrame({'fname':br_mini,'label': [4 for i in range(160)]})])

NameError: ignored

In [8]:
data=pd.concat([data,pd.DataFrame({'fname':dv_mini,'label': [3 for i in range(160)]})])

NameError: ignored

In [0]:
data=pd.concat([data,pd.DataFrame({'fname':mn_mini,'label': [2 for i in range(160)]})])

In [9]:
df = data.sample(frac=1).reset_index(drop=True)

NameError: ignored

In [0]:
df.head()

,fname,label
0,/content/drive/My Drive/ML_data/mn_mini/common...,2
1,/content/drive/My Drive/ML_data/tr_mini/common...,1
2,/content/drive/My Drive/ML_data/br_mini/common...,4
3,/content/drive/My Drive/ML_data/mn_mini/common...,2
4,/content/drive/My Drive/ML_data/br_mini/common...,4


In [0]:
class Config(object):
  
  def __init__(self,
                 lr,epochs,sample_freq=44000, audio_length=5, n_classes=4,
   n_mfcc=20):
    self.sample_freq=sample_freq
    self.n_classes=4
    self.n_mfcc=n_mfcc
    self.audio_length=audio_length;
    self.max_len=audio_length*sample_freq
    self.epochs=epochs
    self.lr=lr
    self.dim = (self.n_mfcc, 1 + int(np.floor(self.max_len/512)), 1)

config=Config(lr=0.0001,epochs=20)    

In [0]:
def prepare_data(df,config):
    X = np.empty(shape=(df.shape[0], config.dim[0], config.dim[1], 1))
    input_length = config.max_len
    for i, fname in enumerate(df.fname):
        file_path =fname
        data, _ = lr.core.load(file_path, sr=config.sample_freq, res_type="kaiser_fast")

        # Random offset / Padding
        if len(data) > input_length:
            max_offset = len(data) - input_length
            offset = np.random.randint(max_offset)
            data = data[offset:(input_length+offset)]
        else:
            if input_length > len(data):
                max_offset = input_length - len(data)
                offset = np.random.randint(max_offset)
            else:
                offset = 0
            data = np.pad(data, (offset, input_length - len(data) - offset), "constant")

        data = lr.feature.mfcc(data,sr=config.sample_freq, n_mfcc=config.n_mfcc)
        data = np.expand_dims(data, axis=-1)
        X[i,] = data
    return X

In [0]:
df.shape[0],config.dim[0],config.dim[1]

(640, 20, 430)

In [0]:
X_train=prepare_data(df,config)

In [0]:
X_train=X_train.squeeze()
x=X_train.reshape(640,-1)
x=torch.tensor(x)
x=x.float()
x=x.to('cuda')

In [0]:
Y_train=np.asarray(df.label)
y=torch.tensor(Y_train)
y=y.to('cuda')

(640,)

In [0]:
from torch import optim
import torch.nn as nn
import torchvision.models as models
import torch

In [0]:
def get_model(lr):
  model=nn.Sequential(nn.Linear(8600,4300),
                      nn.ReLU(),
                      nn.Linear(4300,2150),
                      nn.ReLU(),
                      nn.Linear(2150,1075),
                      nn.ReLU(),
                      nn.Linear(1075,500),
                      nn.ReLU(),
                      nn.Linear(500,200),
                      nn.ReLU(),
                      nn.Linear(200,100),
                      nn.ReLU(),
                      nn.Linear(100,50),
                      nn.ReLU(),
                      nn.Linear(50,5))
                       
  return model,optim.SGD(model.parameters(),lr=lr)

In [0]:
model,opt=get_model(0.001)
model=model.to('cuda')

In [0]:
import torch.nn.functional as F
loss_func=F.cross_entropy

In [0]:
def accuracy(out,yb): return (torch.argmax(out,dim=1)==yb).float().mean()
# x=X_train.reshape(640,430*20).float()
# y=torch.tensor(Y_train)

In [0]:
bs=64
x_tmp=x[:64].float()
pred=model(x_tmp)
pred.shape,pred[0]

(torch.Size([64, 5]),
 tensor([-0.0336,  0.0726,  0.0814, -0.0979, -0.1688], device='cuda:0',
        grad_fn=<SelectBackward>))

In [0]:
y.shape

torch.Size([640])

In [0]:
yb=y[:64]
pred=model(x[:64])
print(pred.shape,yb.shape)
loss_func(pred,yb)

torch.Size([64, 5]) torch.Size([64])


tensor(1.6030, device='cuda:0', grad_fn=<NllLossBackward>)

In [0]:
n=len(Y_train)
bs=64


In [0]:
type(x),x.shape

(torch.Tensor, torch.Size([640, 8600]))

In [0]:
loss_func(model(x.float()), Y_train), accuracy(model(x.float()), Y_train)

In [0]:
X_train.shape

torch.Size([640, 20, 430])

In [0]:
x=torch.tensor(x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [0]:
for epoch in range(40):
    lol=0;
    for i in range((n-1)//64 + 1):
        start_i = i*bs
        end_i = start_i+bs
        xb = x[start_i:end_i].float()
        yb = y[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)
#         print(loss)
        lol+=loss
        loss.backward()
        opt.step()
        opt.zero_grad()
    print(lol/((n-1)//64 + 1))

tensor(0.7387, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7198, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7423, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7211, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6852, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7496, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6451, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6768, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7427, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6382, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6606, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7942, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6248, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7213, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6488, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.6649, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.5366, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.5483, device='cuda:0', grad_fn=<DivBack

In [0]:
loss_func(model(x),y),accuracy(model(x),y)

(tensor(0.3449, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor(0.8906, device='cuda:0'))

In [0]:
for i in range(0,640,64):
  print(i)
  print(i+64)

0
64
64
128
128
192
192
256
256
320
320
384
384
448
448
512
512
576
576
640


In [0]:
from torchvision import models 

In [0]:
print(models.resnet18())

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co